In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import warnings

matplotlib.rcParams['font.family'] = 'Binggrae'
matplotlib.rcParams['axes.unicode_minus'] = False
warnings.filterwarnings('ignore')

In [7]:
df_train = pd.read_csv('train.csv', index_col = 'index')
df_test = pd.read_csv('test.csv', index_col = 'index')

In [8]:
df_train

,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수
index,,,,
0,5011000595017300,2871000192069300,음반,3
1,4148000690043300,5011000264024400,문화컨텐츠,3
2,5011000078068400,1120000007005400,농산물,3
3,4127100048006400,5011000587019400,기타식품,7
4,5011000078068400,2823700010076300,농산물,3
...,...,...,...,...
31679,4471000290087200,5011000213073200,스포츠잡화,3
31680,1129000014045300,5011000319087100,스마트디바이스,4
31681,1129000014045300,5011000263065200,스마트디바이스,6


In [9]:
df_test

,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리
index,,,
0,4167000577042200,5011000435014100,선케어
1,1156000009012200,5011000172034400,구강위생용품
2,4122000363057300,5011000361097300,캠핑
3,5011000436041400,2826000084036400,아웃도어가구
4,4150000241065200,5011000169044300,분유/이유식/아기간식
...,...,...,...
7915,5011000266051200,4623000417038100,농산물
7916,1154500001098300,5011000264055100,문화컨텐츠
7917,5013000610049100,1147000018091400,농산물
